In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

In [9]:
!wget 'https://www.buses.co.uk/open-data/network/current' -O ../additional_data/bus_stops.zip
!mkdir -p ../additional_data/bus_stops
!unzip -o ../additional_data/bus_stops.zip -d ../additional_data/bus_stops
!mv ../additional_data/bus_stops/*/*.xml ../additional_data/bus_stops/

--2021-06-04 17:21:40--  https://www.buses.co.uk/open-data/network/current
Resolving www.buses.co.uk (www.buses.co.uk)... 68.183.252.169
Connecting to www.buses.co.uk (www.buses.co.uk)|68.183.252.169|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://s3-eu-west-1.amazonaws.com/passenger-sources/brightonhove/txc/brightonhove_1622712712.zip [following]
--2021-06-04 17:21:40--  https://s3-eu-west-1.amazonaws.com/passenger-sources/brightonhove/txc/brightonhove_1622712712.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.122.80
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.122.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12554002 (12M) [application/zip]
Saving to: ‘../additional_data/bus_stops.zip’

../additional_data/ 100%[===================>]  11.97M  13.6MB/s    in 0.9s    

2021-06-04 17:21:41 (13.6 MB/s) - ‘../additional_data/bus_stops.zip’ saved [12554002/1

In [1]:
import xml.etree.ElementTree as ET
from shapely.geometry import Point, LineString
import geopandas as gpd
from tqdm import tqdm
import glob
import pickle

In [1]:
def export_stops(file: str) -> list:
    root = ET.parse(file).getroot()
    points = []
    for annotation in root.find('{*}StopPoints'):
        longitude = float(annotation.find('{*}Location').find('{*}Longitude').text)
        latitude = float(annotation.find('{*}Location').find('{*}Latitude').text)
        points.append([latitude, longitude])
    return points

def export_all_stops() -> list:
    all_stops_points = []
    for file in tqdm(glob.glob("../additional_data/bus_stops/*.xml")):
        all_stops_points = all_stops_points + export_stops(file)
    return all_stops_points

with open('../additional_data/bus_stops.pickle', 'wb') as f:
    pickle.dump(export_all_stops(), f, pickle.HIGHEST_PROTOCOL)

100%|██████████| 69/69 [00:27<00:00,  2.50it/s]


In [2]:
def export_routes(file: str) -> list:
    route_sections = ET.parse(file).getroot().find('{*}RouteSections')
    tracks = []
    for route_section in route_sections:
        for route_line in route_section:
            points = []
            for location in route_line.find('{*}Track').find('{*}Mapping').findall('{*}Location'):
                longitude = float(location.find('{*}Longitude').text)
                latitude = float(location.find('{*}Latitude').text)
                points.append(Point(longitude, latitude))
            tracks.append(LineString(points))
    return tracks

def export_all_routes():
    all_routes_lines = []
    for file in tqdm(glob.glob("../additional_data/bus_stops/*.xml")):
        all_routes_lines = all_routes_lines + export_routes(file)
    return gpd.GeoDataFrame(geometry=all_routes_lines)

export_all_routes().to_file("../additional_data/bus_routes.json", driver="GeoJSON")

100%|██████████| 69/69 [00:30<00:00,  2.26it/s]
